In [23]:
with open('../scripts/final_names.txt') as f:
    filenames=f.readlines()
    filenames=[x.strip() for x in filenames]

In [24]:
with open('witnesses_files.txt') as f:
    witnesses_files=f.readlines()
    witnesses_files=[x.strip() for x in witnesses_files]

In [25]:
import numpy as np

In [26]:
inter=np.intersect1d(filenames, witnesses_files)

In [27]:
len(inter)-len(filenames)

-11661

In [28]:
len(inter)-len(witnesses_files)

0

In [29]:
len(witnesses_files)

14195

In [30]:
len(filenames)

25856

In [36]:
faltantes=list(set(filenames)-set(witnesses_files))

In [38]:
len(faltantes)

11661

In [39]:
import re

In [48]:
participants=0
for filename in faltantes:
    with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
        text = f.read()
        if text.find('P A R T I C I P A N T S')>0:
            participants+=1
        elif text.find('Participants')>0:
            participants+=1
        elif text.find('PARTICIPANTS')>0:
            participants+=1
        

In [49]:
participants

735

In [51]:
def es_tipo_witness(texto):
    out=False
    texto=quitar_tabla(texto)
    puntos=texto.find('WITNESS')
    if puntos != -1:
        out=True
    return out

def hay_statement_of(texto):
    out=False
    puntos=texto.find('STATEMENT OF')
    return puntos

def hay_tabla(texto):
    out=False
    puntos=texto.find('.....')
    if puntos != -1:
        bloque="".join(texto[:puntos].split(' '))
        if bloque.find('CONTENTS')!=-1:
            out=True
    return out
        
def mantener_tabla(texto):
    temp = texto
    out=""
    finished = False
    while not finished:
        definidor=temp.find('.....')
        if definidor!=-1:
            out += temp[:definidor+1]
            temp = temp[definidor+1:]
        else:
            finished = True
    return out

def quitar_tabla(texto):
    temp = texto
    finished = False
    while not finished:
        if temp.find('.....')!=-1:
            temp = temp[temp.find('.....')+1:]
        else:
            finished = True
    return temp

def hay_tabla_tipo1(texto):
    if hay_tabla(texto):
        tabla = mantener_tabla(texto)
        return tabla.lower().find("statement of witness") != -1
    return False

def hay_tabla_tipo2(texto):
    if hay_tabla(texto):
        tabla = mantener_tabla(texto)
        return tabla.find("WITNESS") != -1
    return False

def has_witnesses(texto):
    if hay_tabla_tipo1(texto) or hay_tabla_tipo2(texto):
        return True
    elif hay_statement_of(texto) != -1:
        indice = hay_statement_of(texto)
        paragraph = texto[indice-200:indice]
        return (paragraph.lower().find("witness") != -1) or (paragraph.lower().find("testimony") != -1)
    else:
        sin_tabla = quitar_tabla(texto)
        return sin_tabla.find("WITNESS") != -1

In [58]:
for filename in faltantes:
    with open("witnesses_files_faltntes.txt", 'a') as g:
        with open("../data/{}".format(filename), encoding="ISO-8859-1") as f:
            text = f.read()
            if text.lower().find('witness')!=-1:
                print(filename, file=g)